In [1]:
import pandas as pd
import numpy as np
import os
import json
from haversine import haversine
import zipcodes
import datetime

zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/center_of_rentrak_zip.json","r"))
os.getcwd()

'/home/jian/Projects/Savers/TA_creation'

In [2]:
store_zips=pd.ExcelFile("/home/jian/Projects/Savers/TA_creation/store_data/Savers_zips_by_store_for_US_stores_JL_2019-03-08.xlsx")
store_zips.sheet_names

['stroe_list_QCed', 'zips_by_store']

In [3]:
stroe_list_QCed=store_zips.parse("stroe_list_QCed",dtype=str)
zips_by_store=store_zips.parse("zips_by_store",dtype=str)

In [4]:
zips_by_store=zips_by_store.sort_values('location_id').reset_index()
del zips_by_store['index']
stroe_list_QCed=stroe_list_QCed.sort_values('location_id').reset_index()
del stroe_list_QCed['index']

stroe_list_QCed['zip_list_in_5_miles']=stroe_list_QCed['zip_list_in_5_miles'].apply(lambda x: list(eval(x)[0]))

In [5]:
df_TA_zips=pd.DataFrame({"location_id":[stroe_list_QCed['location_id'][0]]*len(stroe_list_QCed['zip_list_in_5_miles'][0]),"zip_cd":stroe_list_QCed['zip_list_in_5_miles'][0],"TA":[1]*len(stroe_list_QCed['zip_list_in_5_miles'][0])},index=[1]*len(stroe_list_QCed['zip_list_in_5_miles'][0]))

TA_counter=1

for i in range(1,len(stroe_list_QCed)):
    intersection_zip=list(set(stroe_list_QCed['zip_list_in_5_miles'][i]).intersection(set(df_TA_zips['zip_cd'].unique().tolist())))
    if len(intersection_zip)==0:
        TA_counter+=1
        df_TA_zips=df_TA_zips.append(pd.DataFrame({"location_id":[stroe_list_QCed['location_id'][i]]*len(stroe_list_QCed['zip_list_in_5_miles'][i]),"zip_cd":stroe_list_QCed['zip_list_in_5_miles'][i],"TA":[TA_counter]*len(stroe_list_QCed['zip_list_in_5_miles'][i])},index=[i]*len(stroe_list_QCed['zip_list_in_5_miles'][i]))).drop_duplicates()
        
    else:
        df_intersection=df_TA_zips[df_TA_zips['zip_cd'].isin(intersection_zip)]
        group_df_intersection=df_intersection.groupby(['TA'])['zip_cd'].count().to_frame().reset_index().sort_values(['zip_cd'],ascending=False)
        selected_TA=group_df_intersection['TA'][0] 
        
        df_TA_zips_0=df_TA_zips[~df_TA_zips['TA'].isin(set(group_df_intersection['TA']))]
        df_TA_zips_1=df_TA_zips[df_TA_zips['TA'].isin(group_df_intersection['TA'].tolist())]
        df_TA_zips_1['TA']=selected_TA
        df_TA_zips=df_TA_zips_0.append(df_TA_zips_1).append(pd.DataFrame({"location_id":[stroe_list_QCed['location_id'][i]]*len(stroe_list_QCed['zip_list_in_5_miles'][i]),"zip_cd":stroe_list_QCed['zip_list_in_5_miles'][i],"TA":[selected_TA]*len(stroe_list_QCed['zip_list_in_5_miles'][i])},index=[i]*len(stroe_list_QCed['zip_list_in_5_miles'][i]))).drop_duplicates()
        
# dict_TA_zips=df_TA_zips.set_index('zip_cd').to_dict()['TA']
dict_TA_store=df_TA_zips.set_index('location_id').to_dict()['TA']
stroe_list_QCed['TA']=stroe_list_QCed['location_id'].apply(lambda x: dict_TA_store[x])

del dict_TA_store

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [6]:
store_cluster=stroe_list_QCed[['location_id','Cluster']].drop_duplicates()

df_TA_zips=pd.merge(df_TA_zips,store_cluster,on="location_id",how="left")
df_TA_zips.head(2)

,TA,location_id,zip_cd,Cluster
0,3,1007,94062,4
1,3,1007,94065,4


In [7]:
def count_unique(x):
    return len(set(x))

test=df_TA_zips.groupby(['TA'])['Cluster'].apply(count_unique).to_frame().reset_index().sort_values("Cluster",ascending=False)

print(test['Cluster'].unique())
test.head(3) # 1 TA only covers 1 cluster

[1]


,TA,Cluster
0,1,1
79,82,1
77,80,1


In [8]:
summary_store=df_TA_zips.groupby("TA")['location_id'].apply(count_unique).to_frame().reset_index().rename(columns={"location_id":"store_count"})
summary_zip=df_TA_zips[['TA','zip_cd']].drop_duplicates().groupby("TA")['zip_cd'].apply(count_unique).to_frame().reset_index().rename(columns={"zip_cd":"zip_count"})

summary_store_list=df_TA_zips.groupby("TA")['location_id'].apply(set).to_frame().reset_index().rename(columns={"storenumber":"store_list"})
summary_zip_list=df_TA_zips[['TA','zip_cd']].drop_duplicates().groupby("TA")['zip_cd'].apply(list).to_frame().reset_index().rename(columns={"zip_cd":"zip_list"})

summary_by_TA=pd.merge(summary_store,summary_zip,on="TA",how="outer")
summary_by_TA=pd.merge(summary_by_TA,summary_store_list,on="TA",how="outer")
summary_by_TA=pd.merge(summary_by_TA,summary_zip_list,on="TA",how="outer")

'''
Good to use the original store zips

TA_Store_zip_list=data.groupby(['TA'])['revised_store_zip'].apply(set).to_frame().reset_index().rename(columns={"revised_store_zip":"store_zip_list"})
summary_by_TA=pd.merge(summary_by_TA,TA_Store_zip_list,on="TA",how="left")
'''
summary_by_TA=pd.merge(summary_by_TA,df_TA_zips[['TA','Cluster']].drop_duplicates(),on="TA",how="outer")
summary_by_TA=summary_by_TA.reset_index()
del summary_by_TA['index']
summary_by_TA=summary_by_TA.reset_index().rename(columns={"index":"ta_number"})
summary_by_TA['ta_number']=summary_by_TA['ta_number']+1

TA_rename_dict=summary_by_TA.set_index("TA").to_dict()['ta_number']
del summary_by_TA['TA']

In [9]:
df_TA_zips.head(2)

,TA,location_id,zip_cd,Cluster
0,3,1007,94062,4
1,3,1007,94065,4


In [10]:
df_TA_zips['ta_number']=df_TA_zips['TA'].apply(lambda x: TA_rename_dict[x])
del df_TA_zips['TA']

stroe_list_QCed['ta_number']=stroe_list_QCed['TA'].apply(lambda x: TA_rename_dict[x])
del stroe_list_QCed['TA']

dict_TA_zips=df_TA_zips.set_index('zip_cd').to_dict()['ta_number']
dict_TA_store=df_TA_zips.set_index('location_id').to_dict()['ta_number']

In [11]:
summary_by_TA.head(2)

,ta_number,store_count,zip_count,location_id,zip_list,Cluster
0,1,3,21,"{1003, 1173, 1113}","[98138, 98131, 98188, 98158, 98178, 98148, 981...",3
1,2,9,56,"{1004, 1021, 1023, 1214, 1125, 1020, 1201, 111...","[98012, 98046, 98020, 98036, 98204, 98087, 980...",3


In [12]:
summary_by_TA['store_count'].sum()

155

In [13]:
summary_by_store_count=summary_by_TA.groupby(['store_count'])['ta_number'].count().to_frame().reset_index().rename(columns={"ta_number":"TA_count"})
summary_by_store_list=summary_by_TA.groupby(['store_count'])['ta_number'].apply(list).to_frame().reset_index().rename(columns={"ta_number":"TA_list"})
summary_by_store_count=pd.merge(summary_by_store_count,summary_by_store_list,on="store_count",how="outer")
summary_by_store_count

,store_count,TA_count,TA_list
0,1,81,"[3, 6, 7, 8, 11, 13, 14, 15, 19, 20, 21, 23, 2..."
1,2,15,"[4, 5, 9, 17, 18, 27, 31, 36, 51, 57, 60, 71, ..."
2,3,5,"[1, 10, 26, 52, 105]"
3,4,1,[12]
4,5,2,"[16, 102]"
5,6,1,[22]
6,9,1,[2]


In [14]:
summary_by_TA['dist_list']=np.nan
summary_by_TA['dist_max']=np.nan
summary_by_TA['dist_median']=np.nan
summary_by_TA['dist_mean']=np.nan
summary_by_TA['dist_min']=np.nan

for index,row in summary_by_TA.iterrows():
    zip_list=row['zip_list']
    dist_list=[]
    for i in range(len(zip_list)):
        for j in range(i+1,len(zip_list)):
            zip_i=zip_list[i]
            zip_j=zip_list[j]
            
            dist=haversine(zip_centers[zip_i],zip_centers[zip_j],miles=True)
            dist_list.append(dist)
    if len(dist_list)>=1:
        summary_by_TA.loc[index, 'dist_list']=str(dist_list)
        summary_by_TA.loc[index,'dist_max']=np.max(dist_list)
        summary_by_TA.loc[index,'dist_median']=np.median(dist_list)
        summary_by_TA.loc[index,'dist_mean']=np.mean(dist_list)
        summary_by_TA.loc[index,'dist_min']=np.min(dist_list)
        
summary_by_TA['ratio_max_to_median']=summary_by_TA['dist_max']/summary_by_TA['dist_median']


In [15]:
df_city_state_in_TA=df_TA_zips[["ta_number","zip_cd"]].drop_duplicates().reset_index()
del df_city_state_in_TA['index']

def city_of_zip(x):
    try:
        city=zipcodes.matching(x)[0]['city']
    except:
        city=np.nan
    return city

def state_of_zip(x):
    try:
        state=zipcodes.matching(x)[0]['state']
    except:
        state=np.nan
    return state
    
df_city_state_in_TA['city']=df_city_state_in_TA['zip_cd'].apply(city_of_zip)
df_city_state_in_TA['state']=df_city_state_in_TA['zip_cd'].apply(state_of_zip)
df_city_state_in_TA['city']=df_city_state_in_TA['city']+" ("+df_city_state_in_TA['state']+")"



In [16]:
df_city_state_in_TA.head(2)

,ta_number,zip_cd,city,state
0,3,94062,REDWOOD CITY (CA),CA
1,3,94065,REDWOOD CITY (CA),CA


In [17]:
zip_DMA=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",dtype=str,skiprows=1)
zip_DMA=zip_DMA.iloc[:,[0,2]]
zip_DMA.columns=['zip_cd','DMA']

df_DMA_in_TA=df_TA_zips[["ta_number","zip_cd"]].drop_duplicates().reset_index()
del df_DMA_in_TA['index']
df_DMA_in_TA=pd.merge(df_DMA_in_TA,zip_DMA,on="zip_cd",how="left")
df_DMA_in_TA.head(2)

,ta_number,zip_cd,DMA
0,3,94062,SAN FRANCISCO-OAK-SAN JOSE
1,3,94065,SAN FRANCISCO-OAK-SAN JOSE


In [18]:
df_city_TA_list=df_city_state_in_TA.groupby(['ta_number'])['city'].apply(set).to_frame().reset_index()
df_city_TA_list=df_city_TA_list.rename(columns={"city":"city_list"})
df_state_TA_list=df_city_state_in_TA.groupby(['ta_number'])['state'].apply(set).to_frame().reset_index()
df_state_TA_list=df_state_TA_list.rename(columns={"state":"state_list"})
df_DMA_TA_list=df_DMA_in_TA.groupby(['ta_number'])['DMA'].apply(set).to_frame().reset_index()
df_DMA_TA_list=df_DMA_TA_list.rename(columns={"DMA":"DMA_list"})


In [19]:
counter_k=1
df_primary_city_state=pd.DataFrame()
df_primary_DMA=pd.DataFrame()
for ta,group in df_city_state_in_TA.groupby(['ta_number']):
    df_city=group.groupby(['city'])['zip_cd'].count().to_frame().reset_index().sort_values(['zip_cd'],ascending=False).reset_index()
    del df_city['index']
    primary_city=df_city['city'][0]
    
    df_state=group.groupby(['state'])['zip_cd'].count().to_frame().reset_index().sort_values(['zip_cd'],ascending=False).reset_index()
    del df_state['index']
    primary_state=df_state['state'][0]
    
    df=pd.DataFrame({"ta_number":ta,"Primary_City":primary_city,"Primary_State":primary_state},index=[counter_k])
    counter_k+=1
    df_primary_city_state=df_primary_city_state.append(df)

    
for ta,group in df_DMA_in_TA.groupby(['ta_number']):
    df_DMA=group.groupby(['DMA'])['zip_cd'].count().to_frame().reset_index().sort_values(['zip_cd'],ascending=False).reset_index()
    del df_DMA['index']
    primary_DMA=df_DMA['DMA'][0]
    
    df=pd.DataFrame({"ta_number":ta,"Primary_DMA":primary_DMA},index=[counter_k])
    counter_k+=1
    df_primary_DMA=df_primary_DMA.append(df)

In [20]:
summary_by_TA=pd.merge(summary_by_TA,df_city_TA_list,on="ta_number",how="left")
summary_by_TA=pd.merge(summary_by_TA,df_state_TA_list,on="ta_number",how="left")
summary_by_TA=pd.merge(summary_by_TA,df_DMA_TA_list,on="ta_number",how="left")

summary_by_TA=pd.merge(summary_by_TA,df_primary_city_state,on="ta_number",how="left")
summary_by_TA=pd.merge(summary_by_TA,df_primary_DMA,on="ta_number",how="left")


In [21]:
writer=pd.ExcelWriter("/home/jian/Projects/Savers/TA_creation/TA_output/Savers_US_TA_1st_Version_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")

stroe_list_QCed.to_excel(writer,"Store_list",index=False)
summary_by_TA.to_excel(writer,"Initial_TA_Summary",index=False)
summary_by_store_count.to_excel(writer,"TA_Summary_by_store_count",index=False)
df_TA_zips.to_excel(writer,"TA_zips",index=False)

writer.save()